In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd 

In [2]:
import cv2
import numpy as np

In [3]:
data_path = "/kaggle/input/binary-aperio-inversegaussian/IGGAperio"

In [4]:
all_data = os.listdir(data_path)
image_names = [image.split(".")[0] for image in  all_data if image.split(".")[-1]=="tiff"]

In [5]:
class SegmentationDataset(Dataset):
    
    def __init__(self,image_path, images,feature_extractor):
        """_summary_

        Args:
            image_path (string): _Path of the Images_
            images (_type_): _Array of name of images_
            feature_extractor (_type_): _Image feature extractor for Segformer_
        """
        
        self.image_path = image_path
        self.images = images
        self.feature_extractor = feature_extractor
        
        
    def __len__(self):
        """_summary_

        Returns:
            _int_: _length of dataset_
        """
        return len(self.images)
    
    def __getitem__(self,idx):


        image_name = self.images[idx]
        tr = cv2.imread(self.image_path +"/" +image_name +"_mitosis.jpg")
        lower = np.array([0, 3, 240])
        upper = np.array([255, 255, 255])
        segmentation_mask = cv2.inRange(tr, lower, upper)
        image = Image.open(self.image_path +"/" +image_name +".tiff")
        #segmentation_mask = Image.open(self.image_path +"/" +image_name +"_mitosis.jpg")
        image = image.convert('RGB')
        
        ## Encoding the data using feature extractor
        encodings = self.feature_extractor(image,segmentation_mask,return_tensors = "pt")
        
        ## Removing the dimension of the batch
        for k,v in encodings.items():
            encodings[k] = v.squeeze_()  
            
        return encodings

In [6]:
from transformers import SegformerFeatureExtractor


feature_extractor = SegformerFeatureExtractor()

In [7]:
train_dataset = SegmentationDataset(data_path,image_names,feature_extractor)

In [8]:
len(train_dataset[0]['labels'].squeeze().unique())

1

In [9]:
for i in train_dataset:
    if (len(i['labels'].squeeze().unique()) != 1):
        print(i['labels'].squeeze().unique())
        break

tensor([  0, 255])


In [10]:
for k,v in train_dataset[0].items():
    print(v.size())

torch.Size([3, 512, 512])
torch.Size([512, 512])


In [11]:
id2label = {0:0,1:255}
label2id = {0:0,255:1}

In [12]:
from transformers import SegformerForSemanticSegmentation
model_name = "nvidia/mit-b0"
model = SegformerForSemanticSegmentation.from_pretrained(model_name,
                                                         num_labels=2, 
                                                         id2label=id2label, 
                                                         label2id=label2id,
)

Downloading:   0%|          | 0.00/68.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.weight', 'decode_head.

Intialising Dataloader

In [13]:
from torch.utils.data import DataLoader

In [14]:
batch_size = 16

In [15]:
len(image_names)

989

In [16]:
import random
random.shuffle(image_names)

In [17]:
train_dataset = SegmentationDataset(data_path,image_names[:900],feature_extractor) ## Taking 100 Images for training for now
test_dataset =  SegmentationDataset(data_path,image_names[900:989],feature_extractor)

In [18]:
len(test_dataset)

89

In [19]:
test_dataloader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [20]:
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [21]:
len(test_dataloader)

6

Finetuning

In [22]:
from datasets import load_metric

metric = load_metric("mean_iou")
#metric.compute()

In [23]:
path = "/kaggle/working/Aperio_InverseGaussian_Binary.pt"
path1 = "/kaggle/working/Aperio_InverseGaussian_Binary_loss.pt"

In [24]:
import torch
from torch import nn
#from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm


# define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006)
# move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("done")

done


In [25]:
maxx = 100
model.train()
for epoch in range(50):  # loop over the dataset multiple times
    metric = load_metric("mean_iou")
    
    print("Epoch:", epoch)
    for idx, batch in enumerate(tqdm(train_dataloader)):
        # get the inputs;
        pixel_values = batch["pixel_values"].cuda()
        labels = batch["labels"].cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits
        
        loss.backward()
        optimizer.step()

        # evaluate
        
    with torch.no_grad():
        
        for idx, batch in enumerate(tqdm(test_dataloader)):
    # get the inputs;
            pixel_values1 = batch["pixel_values"].cuda()
            labels1 = batch["labels"].cuda()
            outputs1 = model(pixel_values=pixel_values1, labels=labels1)
            loss, logits = outputs1.loss, outputs1.logits
            upsampled_logits = nn.functional.interpolate(logits, size=labels1.shape[-2:], mode="bilinear", align_corners=False)
            predicted = upsampled_logits.argmax(dim=1)
            metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels1.detach().cpu().numpy())
        
        # note that the metric expects predictions + labels as numpy arrays

        # let's print loss and metrics every 100 batches
        #predictions=predicted.detach().cpu().numpy()

        #metrics = metric._compute(predictions=predicted.detach().cpu().numpy(), references=labels1.detach().cpu().numpy(),num_labels=len(id2label), 
                               #ignore_index=255,
                               #reduce_labels=False, # we've already reduced the labels before)
                                # )
        metrics = metric.compute(num_labels=len(id2label), ignore_index=255, reduce_labels=False)
        ars = loss.item()
        print("Loss:", ars)
        print("Mean_iou:", metrics["mean_iou"])
        print("Mean accuracy:", metrics["mean_accuracy"])

        if maxx > ars:
            maxx = ars
            torch.save(model, path1)

Epoch: 0


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

/root/.cache/huggingface/modules/datasets_modules/metrics/mean_iou/d4add40cf977cdd73590b5873fa830f3f13adb678f6777a29fb07b7c81d14342/mean_iou.py:259: RuntimeWarning: invalid value encountered in true_divide
  acc = total_area_intersect / total_area_label


Loss: 0.3202272057533264
Mean_iou: 0.4996718440759765
Mean accuracy: 0.999343688151953
Epoch: 1


...

  0%|          | 0/6 [00:00<?, ?it/s]

Loss: 0.0013408998493105173
Mean_iou: 1.0
Mean accuracy: 1.0
Epoch: 49


  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Loss: 0.003542681923136115
Mean_iou: 1.0
Mean accuracy: 1.0
